# Pandas 1.3

**RECAP.** Use of pandas can generally be divided into four. The following covers the general uses of pandas in data wrangling.
- Loading the data
- Exploring the data
- Cleaning the data
- Preparing data for visualization


For this notebook, focus will be on **preparing the data for visualization**. Since we'll be working on combining datasets and aggregations, a new dataset is prepared below. We will be merging this new data

In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:,.4f}'.format)

In [2]:
ir_df = pd.read_csv('data/PHIR71FL-labeled.csv', dtype = 'unicode')
print("Initial shape: ", ir_df.shape)

ir_df.dropna(axis = 1, how = 'all', inplace = True)
ir_df = ir_df.loc[:,~ir_df.columns.str.contains('\$')]
print("Final shape: ", ir_df.shape)

ir_df.to_csv('data/PHIR71FL-labeled-clean.csv', index = False)

Initial shape:  (25074, 5731)
Final shape:  (25074, 494)


In [3]:
with open(f'data/PHIR71FL-data-dictionary.json', 'r') as file:
    ir_col_dict = json.load(file)
    
display(ir_col_dict)

{'CASEID': 'Case Identification',
 'V000': 'Country code and phase',
 'V001': 'Cluster number',
 'V002': 'Household number',
 'V003': "Respondent's line number",
 'V004': 'Ultimate area unit',
 'V005': "Women's individual sample weight (6 decimals)",
 'V006': 'Month of interview',
 'V007': 'Year of interview',
 'V008': 'Date of interview (CMC)',
 'V008A': 'Date of interview Century Day Code (CDC)',
 'V009': "Respondent's month of birth",
 'V010': "Respondent's year of birth",
 'V011': 'Date of birth (CMC)',
 'V012': "Respondent's current age",
 'V013': 'Age in 5-year groups',
 'V014': 'Completeness of age information',
 'V015': 'Result of individual interview',
 'V016': 'Day of interview',
 'V017': 'NA - CMC start of calendar',
 'V018': 'Row of month of interview',
 'V019': 'Length of calendar',
 'V019A': 'Number of calendar columns',
 'V020': 'Ever-married sample',
 'V021': 'Primary sampling unit',
 'V022': 'Sample strata for sampling errors',
 'V024': 'Region',
 'V025': 'Type of plac

--------------------------------
## Loading the datasets & their dictionaries

Let's start by loading the clean datasets: ```PHHR71FL-labeled-clean.csv``` and ```PHIR71FL-labeled-clean.csv```

In [4]:
# Read the two datasets
hr_df = pd.read_csv('data/PHHR71FL-labeled-clean.csv')
ir_df = pd.read_csv('data/PHIR71FL-labeled-clean.csv')

<ipython-input-4-377a3c7cbf9d>:2: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  hr_df = pd.read_csv('data/PHHR71FL-labeled-clean.csv')
<ipython-input-4-377a3c7cbf9d>:3: DtypeWarning: Columns (60,70,211,266,330,331,475,476,477,485) have mixed types. Specify dtype option on import or set low_memory=False.
  ir_df = pd.read_csv('data/PHIR71FL-labeled-clean.csv')


In [5]:
with open(f'data/PHHR71FL-data-dictionary.json', 'r') as file:
    hr_col_dict = json.load(file)
    
display(hr_col_dict)

{'HHID': 'Case Identification',
 'HV000': 'Country code and phase',
 'HV001': 'Cluster number',
 'HV002': 'Household number',
 'HV003': "Respondent's line number (answering Household questionnaire)",
 'HV004': 'Ultimate area unit',
 'HV005': 'Household sample weight (6 decimals)',
 'HV006': 'Month of interview',
 'HV007': 'Year of interview',
 'HV008': 'Date of interview (CMC)',
 'HV008A': 'Date of interview Century Day Code (CDC)',
 'HV009': 'Number of household members',
 'HV010': 'Number of eligible women in household',
 'HV011': 'NA - Number of eligible men in household',
 'HV012': 'Number of de jure members',
 'HV013': 'Number of de facto members',
 'HV014': 'Number of children 5 and under (de jure)',
 'HV015': 'Result of household interview',
 'HV016': 'Day of interview',
 'HV017': 'Number of visits',
 'HV018': 'Interviewer identification',
 'HV019': 'NA - Keyer identification',
 'HV020': 'Ever-married sample',
 'HV021': 'Primary sampling unit',
 'HV022': 'Sample strata for sampl

In [6]:
display(ir_col_dict)

{'CASEID': 'Case Identification',
 'V000': 'Country code and phase',
 'V001': 'Cluster number',
 'V002': 'Household number',
 'V003': "Respondent's line number",
 'V004': 'Ultimate area unit',
 'V005': "Women's individual sample weight (6 decimals)",
 'V006': 'Month of interview',
 'V007': 'Year of interview',
 'V008': 'Date of interview (CMC)',
 'V008A': 'Date of interview Century Day Code (CDC)',
 'V009': "Respondent's month of birth",
 'V010': "Respondent's year of birth",
 'V011': 'Date of birth (CMC)',
 'V012': "Respondent's current age",
 'V013': 'Age in 5-year groups',
 'V014': 'Completeness of age information',
 'V015': 'Result of individual interview',
 'V016': 'Day of interview',
 'V017': 'NA - CMC start of calendar',
 'V018': 'Row of month of interview',
 'V019': 'Length of calendar',
 'V019A': 'Number of calendar columns',
 'V020': 'Ever-married sample',
 'V021': 'Primary sampling unit',
 'V022': 'Sample strata for sampling errors',
 'V024': 'Region',
 'V025': 'Type of plac

---------------------------------------
## Combining dataframes

It makes sense to combine all datasets in one dataframe (if possible) prior further wrangling it for visualization & analysis.

There are a few ways to combine two dataframes:
- By using ```pd.concat()```. Used to **append one or more dataframes below or sideways** the other depending on whether the axis option is set to 0 or 1
- By using ```.join()```. Used to **merge two dataframes on the basis of their indices**
- By using ```.merge()```. Used to **combine two or more dataframes on the basis of values of common columns** (indices can also be used)

Read more about them from [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).

Given the different ways to combine dataframes, how do we decide our method for combining our two dataframes?

Let's recall what we know about our datasets...

How big is ```hr_df``` and ```ir_df```?

In [7]:
# Check size of hr_df
hr_df.shape

(27496, 338)

In [8]:
# Check size of ir_df
ir_df.shape

(25074, 494)

What are the columns for each dataframe?

In [9]:
# Check column names
hr_df.columns

Index(['HHID', 'HV000', 'HV001', 'HV002', 'HV003', 'HV004', 'HV005', 'HV006',
       'HV007', 'HV008',
       ...
       'SH512D', 'SH512E', 'SH512F', 'SH512G', 'SH512X', 'SH512Z', 'SH513',
       'SH514', 'SH515', 'SHNUMDV'],
      dtype='object', length=338)

In [10]:
ir_df.columns

Index(['CASEID', 'V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006',
       'V007', 'V008',
       ...
       'S717', 'S723', 'S815E', 'S904L', 'S933', 'S1107A', 'S1107B', 'SEDUC',
       'SPROV', 'SPEDUC'],
      dtype='object', length=494)

### Using .merge()

Based on these two checks, we know that ```hr_df``` has more rows than ```ir_df```. From the data dictionaries, we also know that both datasets have cluster numbers and household numbers. The DHS Program suggests that we use these columns to merge the two datasets: [link](https://dhsprogram.com/data/Merging-Datasets.cfm#CP_JUMP_8050).

From what we've learned a few moments ago, use of ```.merge()``` fits our use case. Before trying to merge the two dataframes, let's inspect the values of the cluster number and household number columns for each dataframe since we'll be using them as basis.

Since we'll use these columns as the reference columns for merging, the expectation is that their values align. Meaning, the values of the cluster and household numbers existing in both dataframes should match.

In [11]:
# How many unique cluster numbers are in hr_df?
hr_df['HV001'].nunique()

1249

In [12]:
# How many unique household numbers are in hr_df?
hr_df['HV002'].nunique()

59

In [13]:
# How many unique combination of cluster and household numbers are in hr_df?
len(hr_df[['HV001','HV002']].drop_duplicates(keep = 'first'))

27496

In [14]:
# How many unique cluster numbers are in ir_df?
ir_df['V001'].nunique()

1248

In [15]:
# How many unique household numbers are in ir_df?
ir_df['V002'].nunique()

53

In [16]:
# How many unique combination of cluster and household numbers are in ir_df?
len(ir_df[['V001','V002']].drop_duplicates(keep = 'first'))

18644

Let's try to merge our dataframes...

There are multiple arguments set for ```.merge()```. Ones we'll be using are as follows.
- ```right```: DataFrame or named Series to merge with
- ```how```: Type of merge to be performed. Can be {'left', 'right', 'outer', 'inner'}, default is 'inner'
- ```on```: Column or index level names to join on. These must be found in both dataframes. If the column names of the reference columns are different for the two datasets we can also use the `right_on` and `left_on` arguments

Check out other arguments by pressing shift + tab.

In [17]:
# Let's merge hr_df to ir_df
merged_df = ir_df.merge(hr_df, how = 'left', left_on = ['V001','V002'], right_on = ['HV001','HV002'])
merged_df.head()

,CASEID,V000,V001,V002,V003,V004,V005,V006,V007,V008,V008A,V009,V010,V011,V012,V013,V014,V015,V016,V018,V019,V019A,V020,V021,V022,V024,V025,V027,V028,V030,V034,V042,V044,V045A,V045B,V045C,V046,V023,V101,V102,V104,V105,V105A,V106,V107,V113,V115,V116,V119,V120,V121,V122,V123,V124,V125,V127,V128,V129,V130,V131,V133,V135,V136,V137,V138,V139,V140,V149,V150,V151,V152,V153,AWFACTT,AWFACTU,AWFACTR,AWFACTE,AWFACTW,V157,V158,V159,V160,V161,V169A,V169B,V170,V171A,V171B,V190,V191,V190A,V191A,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V235,V237,V238,V244,V301,V302A,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V325A,V326,V327,V337,V361,V362,V363,V364,V367,V376,V376A,V379,V380,V384A,V384B,V384C,V384D,V393,V393A,V394,V395,V3A00Y,V3A00Z,V3A01,V3A02,V3A03,V3A04,V3A05,V3A06,V3A07,V3A08A,V3A08B,V3A08C,V3A08D,V3A08E,V3A08F,V3A08G,V3A08H,V3A08I,V3A08J,V3A08K,V3A08L,V3A08M,V3A08N,V3A08P,V3A08Q,V3A08R,V3A08S,V3A08T,V3A08U,V3A08V,V3A08X,V3A08Z,V401,V404,V405,V406,V415,V416,V417,V418,V418A,V426,V447,V463A,V463B,V463C,V463D,V463E,V463F,V463G,V463H,V463I,V463X,V463Z,V463AA,V463AB,V464,V465,V467B,V467C,V467D,V467F,V468,V501,V502,V503,V504,V507,V508,V509,V510,V511,V512,V513,V525,V527,V528,V529,V530,V531,V532,V535,V536,V537,V602,V603,V604,V605,V613,V614,V616,V621,V623,V624,V625,V626,V625A,V626A,V627,V628,V629,V632,V632A,V633B,V701,V702,V704,V704A,V705,V714,V714A,V715,V716,V717,V719,V729,V730,V731,V732,V739,V741,V743A,V743B,V743D,V743F,V744A,V744B,V744C,V744D,V744E,V745A,V745B,V746,V750,V751,V754CP,V754DP,V754JP,V754WP,V756,V761,V762,V763A,V763B,V763C,V766A,V766B,V767A,V774A,V774B,V774C,V777A,V781,V783,V784A,V784B,V784C,V784D,V784E,V784F,V784K,V784L,V784M,V784N,V784O,V784X,V791A,V822,V823,V824,V825,V826A,V828,V829,V833A,V834A,V836,V850A,V850B,V852A,V853A,V857A,V857B,V857C,V857D,V858,V801,V802,V803,V804,V805,V811,V812,V813,V814,V815A,V815B,V815C,D005,D101A,D101B,D101C,D101D,D101E,D102,D103A,D103B,D103C,D103D,D103E,D103F,D103G,D104,D105A,D105B,D105C,D105D,D105E,D105F,D105H,D105I,D105J,D105K,D106,D107,D108,D109,D110A,D110B,D110D,D110E,D110F,D110G,D111,D112,D112A,D113,D114,D115B,D115C,D115D,D115F,D115G,D115K,D115L,D115O,D115P,D115Q,D115V,D115W,D115X,D115Y,D115XE,D117A,D118A,D118B,D118C,D118D,D118F,D118G,D118J,D118K,D118L,D118O,D118P,D118Q,D118V,D118W,D118X,D118Y,D118XE,D119H,D119I,D119J,D119K,D119U,D119X,D119Y,D119XB,D119XD,D119XE,D119XF,D119XG,D119XH,D121,D122A,D122B,D122C,D124,D125,D126,D127,D128,D129,D130A,D130B,D130C,S1229AA,S1229AB,S1229AC,S1229BA,S1229BB,S1229BC,S1229BD,S1229BE,S1229BF,S1229BG,S102A,S104BA,S104BB,S104BC,S104BD,S104BZ,S104C,S104P,S108L,S109,S207C,S207D,S208,S239A,S305A,S306A,S308A,S308B,S308C,S705B,S705C,S705D,S717,S723,S815E,S904L,S933,S1107A,S1107B,SEDUC,SPROV,SPEDUC,HHID,HV000,HV001,HV002,HV003,HV004,HV005,HV006,HV007,HV008,HV008A,HV009,HV010,HV012,HV013,HV014,HV015,HV016,HV017,HV018,HV020,HV021,HV022,HV023,HV024,HV025,HV027,HV028,HV030,HV042,HV044,HV045A,HV045B,HV045C,HV046,HV801,HV802,HV803,HV201,HV202,HV201A,HV204,HV205,HV206,HV207,HV208,HV209,HV210,HV211,HV212,HV213,HV214,HV215,HV216,HV217,HV218,HV219,HV220,HV221,HV225,HV226,HV230A,HV230B,HV232,HV232B,HV232Y,HV235,HV237,HV237A,HV237B,HV237C,HV237D,HV237E,HV237F,HV237X,HV237Z,HV238,HV238A,HV241,HV242,HV243A,HV243B,HV243C,HV243D,HV243E,HV244,HV245,HV246,HV246A,HV246B,HV246C,HV246D,HV246E,HV246F,HV246G,HV252,HV270,HV271,HV270A,HV271A,SHUSN,SHSN,SHPROV,SH109B,SH109C,SH121G,SH121H,SH121I,SH121J,SH121K,SH122H,SH123A,SH123B,SH124A,SH201,SH202,SH206,SH207,SH220,SH221A,SH301A,SH301B,SH301C,SH301D,SH301E,SH301F,SH301G,SH301H,SH301I,SH301J,SH301K,SH301L,SH301M,SH301N,SH301O,SH301X,SH301Z,SH302A,SH302B,SH302C,SH302D,SH302E,SH302F,SH302G,SH302H,SH302I,SH302J,SH302K,SH302L,SH302M,SH302N,SH302O,SH302X,SH302Y,SH302Z,SH303,SH304,SH305,SH306,SH307,SH308A,SH308B,SH308C,SH308D,SH308E,SH308F,SH308G,SH308H,SH308X,SH401A,SH401B,SH401C,SH401D,SH401E,SH401F,SH401G,SH401H,SH401I,SH401J,SH401K,SH401L,SH401M,SH401N,

In [18]:
# Check the columns of the merged dataset
merged_df.columns

Index(['CASEID', 'V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006',
       'V007', 'V008',
       ...
       'SH512D', 'SH512E', 'SH512F', 'SH512G', 'SH512X', 'SH512Z', 'SH513',
       'SH514', 'SH515', 'SHNUMDV'],
      dtype='object', length=832)

In [19]:
merged_df.shape

(25074, 832)

To use the `on` argument, we can also rename `HV001 to V001` and `HV002 to V002` in `hr_df` OR rename `V001 to HV001` and `V002 to HV002` in the `ir_df` prior merging.

Let's save the ```merged_df``` to a csv file named ```PH-HRIR-merged.csv```.

In [20]:
# Save to csv
merged_df.to_csv('data/PH-HRIR-merged.csv', index = False)

-----------------------------
Now that we're done combining dataframes, we can play around the ```merged_df``` and come up with versions of it that are good to visualize later on. We can either **create new columns to visualize** or **do aggregations of the current information available in the merged dataset**.

## Creating new columns 

The simplest way to add a new column to an existing dataframe is to index the dataframe with the new column name and assign a value to it. This is where use of ```numpy``` methods might come in handy later on.

**Example.** Let's create columns recoding the values of the following. 
- `SH301A`: LGU Health programs you are  aware of - Free medical consultation
- `SH301B`: LGU Health programs you are  aware of - Free medicines
- `SH301C`: LGU Health programs you are  aware of - Free laboratory/blood tests
- `SH301D`: LGU Health programs you are  aware of - Free immunization/vaccines
- `SH301E`: LGU Health programs you are  aware of - Free test screening for diseases (example cancer, hiv, etc)
- `SH301F`: LGU Health programs you are  aware of - Free minor surgeries (example eye cataract, circumision)
- `SH301G`: LGU Health programs you are  aware of - Free dental check up
- `SH301H`: LGU Health programs you are  aware of - Free dental extraction
- `SH301I`: LGU Health programs you are  aware of - Family planning programs
- `SH301J`: LGU Health programs you are  aware of - Health and wellness programs
- `SH301K`: LGU Health programs you are  aware of - Medical assistance on hospitalization/laboratory tests
- `SH301L`: LGU Health programs you are  aware of - Mental health programs
- `SH301M`: LGU Health programs you are  aware of - Free medical assistance on hospitalization/laboratory tests
- `SH301N`: LGU Health programs you are  aware of - Free deworming
- `SH301O`: LGU Health programs you are  aware of - Free prenatal care
- `SH301X`: LGU Health programs you are  aware of - Other

We'll start by inspecting the original values...

In [21]:
# Create a list of columns for SH301, not including SH301Z (Don't know)
sh301_cols = [x for x in merged_df.columns if 'SH301' in x and x != 'SH301Z']

In [22]:
# Inspect unique values
for col in sh301_cols:
    print(col, merged_df[col].unique())
    print('-----'*10)

SH301A ['No' 'Yes']
--------------------------------------------------
SH301B ['No' 'Yes']
--------------------------------------------------
SH301C ['No' 'Yes']
--------------------------------------------------
SH301D ['No' 'Yes']
--------------------------------------------------
SH301E ['No' 'Yes']
--------------------------------------------------
SH301F ['No' 'Yes']
--------------------------------------------------
SH301G ['No' 'Yes']
--------------------------------------------------
SH301H ['No' 'Yes']
--------------------------------------------------
SH301I ['No' 'Yes']
--------------------------------------------------
SH301J ['No' 'Yes']
--------------------------------------------------
SH301K ['No' 'Yes']
--------------------------------------------------
SH301L ['No' 'Yes']
--------------------------------------------------
SH301M ['No' 'Yes']
--------------------------------------------------
SH301N ['No' 'Yes']
--------------------------------------------------
SH301O

In [23]:
# Now, create a new column with recoded sh301 values: Yes - 1, No - 0
merged_df['SH301A-recoded'] = merged_df['SH301A'].replace({'Yes':1, 'No': 0})
merged_df.head()

,CASEID,V000,V001,V002,V003,V004,V005,V006,V007,V008,V008A,V009,V010,V011,V012,V013,V014,V015,V016,V018,V019,V019A,V020,V021,V022,V024,V025,V027,V028,V030,V034,V042,V044,V045A,V045B,V045C,V046,V023,V101,V102,V104,V105,V105A,V106,V107,V113,V115,V116,V119,V120,V121,V122,V123,V124,V125,V127,V128,V129,V130,V131,V133,V135,V136,V137,V138,V139,V140,V149,V150,V151,V152,V153,AWFACTT,AWFACTU,AWFACTR,AWFACTE,AWFACTW,V157,V158,V159,V160,V161,V169A,V169B,V170,V171A,V171B,V190,V191,V190A,V191A,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V235,V237,V238,V244,V301,V302A,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V325A,V326,V327,V337,V361,V362,V363,V364,V367,V376,V376A,V379,V380,V384A,V384B,V384C,V384D,V393,V393A,V394,V395,V3A00Y,V3A00Z,V3A01,V3A02,V3A03,V3A04,V3A05,V3A06,V3A07,V3A08A,V3A08B,V3A08C,V3A08D,V3A08E,V3A08F,V3A08G,V3A08H,V3A08I,V3A08J,V3A08K,V3A08L,V3A08M,V3A08N,V3A08P,V3A08Q,V3A08R,V3A08S,V3A08T,V3A08U,V3A08V,V3A08X,V3A08Z,V401,V404,V405,V406,V415,V416,V417,V418,V418A,V426,V447,V463A,V463B,V463C,V463D,V463E,V463F,V463G,V463H,V463I,V463X,V463Z,V463AA,V463AB,V464,V465,V467B,V467C,V467D,V467F,V468,V501,V502,V503,V504,V507,V508,V509,V510,V511,V512,V513,V525,V527,V528,V529,V530,V531,V532,V535,V536,V537,V602,V603,V604,V605,V613,V614,V616,V621,V623,V624,V625,V626,V625A,V626A,V627,V628,V629,V632,V632A,V633B,V701,V702,V704,V704A,V705,V714,V714A,V715,V716,V717,V719,V729,V730,V731,V732,V739,V741,V743A,V743B,V743D,V743F,V744A,V744B,V744C,V744D,V744E,V745A,V745B,V746,V750,V751,V754CP,V754DP,V754JP,V754WP,V756,V761,V762,V763A,V763B,V763C,V766A,V766B,V767A,V774A,V774B,V774C,V777A,V781,V783,V784A,V784B,V784C,V784D,V784E,V784F,V784K,V784L,V784M,V784N,V784O,V784X,V791A,V822,V823,V824,V825,V826A,V828,V829,V833A,V834A,V836,V850A,V850B,V852A,V853A,V857A,V857B,V857C,V857D,V858,V801,V802,V803,V804,V805,V811,V812,V813,V814,V815A,V815B,V815C,D005,D101A,D101B,D101C,D101D,D101E,D102,D103A,D103B,D103C,D103D,D103E,D103F,D103G,D104,D105A,D105B,D105C,D105D,D105E,D105F,D105H,D105I,D105J,D105K,D106,D107,D108,D109,D110A,D110B,D110D,D110E,D110F,D110G,D111,D112,D112A,D113,D114,D115B,D115C,D115D,D115F,D115G,D115K,D115L,D115O,D115P,D115Q,D115V,D115W,D115X,D115Y,D115XE,D117A,D118A,D118B,D118C,D118D,D118F,D118G,D118J,D118K,D118L,D118O,D118P,D118Q,D118V,D118W,D118X,D118Y,D118XE,D119H,D119I,D119J,D119K,D119U,D119X,D119Y,D119XB,D119XD,D119XE,D119XF,D119XG,D119XH,D121,D122A,D122B,D122C,D124,D125,D126,D127,D128,D129,D130A,D130B,D130C,S1229AA,S1229AB,S1229AC,S1229BA,S1229BB,S1229BC,S1229BD,S1229BE,S1229BF,S1229BG,S102A,S104BA,S104BB,S104BC,S104BD,S104BZ,S104C,S104P,S108L,S109,S207C,S207D,S208,S239A,S305A,S306A,S308A,S308B,S308C,S705B,S705C,S705D,S717,S723,S815E,S904L,S933,S1107A,S1107B,SEDUC,SPROV,SPEDUC,HHID,HV000,HV001,HV002,HV003,HV004,HV005,HV006,HV007,HV008,HV008A,HV009,HV010,HV012,HV013,HV014,HV015,HV016,HV017,HV018,HV020,HV021,HV022,HV023,HV024,HV025,HV027,HV028,HV030,HV042,HV044,HV045A,HV045B,HV045C,HV046,HV801,HV802,HV803,HV201,HV202,HV201A,HV204,HV205,HV206,HV207,HV208,HV209,HV210,HV211,HV212,HV213,HV214,HV215,HV216,HV217,HV218,HV219,HV220,HV221,HV225,HV226,HV230A,HV230B,HV232,HV232B,HV232Y,HV235,HV237,HV237A,HV237B,HV237C,HV237D,HV237E,HV237F,HV237X,HV237Z,HV238,HV238A,HV241,HV242,HV243A,HV243B,HV243C,HV243D,HV243E,HV244,HV245,HV246,HV246A,HV246B,HV246C,HV246D,HV246E,HV246F,HV246G,HV252,HV270,HV271,HV270A,HV271A,SHUSN,SHSN,SHPROV,SH109B,SH109C,SH121G,SH121H,SH121I,SH121J,SH121K,SH122H,SH123A,SH123B,SH124A,SH201,SH202,SH206,SH207,SH220,SH221A,SH301A,SH301B,SH301C,SH301D,SH301E,SH301F,SH301G,SH301H,SH301I,SH301J,SH301K,SH301L,SH301M,SH301N,SH301O,SH301X,SH301Z,SH302A,SH302B,SH302C,SH302D,SH302E,SH302F,SH302G,SH302H,SH302I,SH302J,SH302K,SH302L,SH302M,SH302N,SH302O,SH302X,SH302Y,SH302Z,SH303,SH304,SH305,SH306,SH307,SH308A,SH308B,SH308C,SH308D,SH308E,SH308F,SH308G,SH308H,SH308X,SH401A,SH401B,SH401C,SH401D,SH401E,SH401F,SH401G,SH401H,SH401I,SH401J,SH401K,SH401L,SH401M,SH401N,

#### <font color= 'pink'> **Practice:**</font> Make new columns with recoded values for other items in the `sh301_cols`

In [24]:
# Insert code here

for col in sh301_cols:
    merged_df[col + '-recoded'] = merged_df[col].replace({'Yes':1, 'No': 0})

merged_df.head()

,CASEID,V000,V001,V002,V003,V004,V005,V006,V007,V008,V008A,V009,V010,V011,V012,V013,V014,V015,V016,V018,V019,V019A,V020,V021,V022,V024,V025,V027,V028,V030,V034,V042,V044,V045A,V045B,V045C,V046,V023,V101,V102,V104,V105,V105A,V106,V107,V113,V115,V116,V119,V120,V121,V122,V123,V124,V125,V127,V128,V129,V130,V131,V133,V135,V136,V137,V138,V139,V140,V149,V150,V151,V152,V153,AWFACTT,AWFACTU,AWFACTR,AWFACTE,AWFACTW,V157,V158,V159,V160,V161,V169A,V169B,V170,V171A,V171B,V190,V191,V190A,V191A,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V235,V237,V238,V244,V301,V302A,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V325A,V326,V327,V337,V361,V362,V363,V364,V367,V376,V376A,V379,V380,V384A,V384B,V384C,V384D,V393,V393A,V394,V395,V3A00Y,V3A00Z,V3A01,V3A02,V3A03,V3A04,V3A05,V3A06,V3A07,V3A08A,V3A08B,V3A08C,V3A08D,V3A08E,V3A08F,V3A08G,V3A08H,V3A08I,V3A08J,V3A08K,V3A08L,V3A08M,V3A08N,V3A08P,V3A08Q,V3A08R,V3A08S,V3A08T,V3A08U,V3A08V,V3A08X,V3A08Z,V401,V404,V405,V406,V415,V416,V417,V418,V418A,V426,V447,V463A,V463B,V463C,V463D,V463E,V463F,V463G,V463H,V463I,V463X,V463Z,V463AA,V463AB,V464,V465,V467B,V467C,V467D,V467F,V468,V501,V502,V503,V504,V507,V508,V509,V510,V511,V512,V513,V525,V527,V528,V529,V530,V531,V532,V535,V536,V537,V602,V603,V604,V605,V613,V614,V616,V621,V623,V624,V625,V626,V625A,V626A,V627,V628,V629,V632,V632A,V633B,V701,V702,V704,V704A,V705,V714,V714A,V715,V716,V717,V719,V729,V730,V731,V732,V739,V741,V743A,V743B,V743D,V743F,V744A,V744B,V744C,V744D,V744E,V745A,V745B,V746,V750,V751,V754CP,V754DP,V754JP,V754WP,V756,V761,V762,V763A,V763B,V763C,V766A,V766B,V767A,V774A,V774B,V774C,V777A,V781,V783,V784A,V784B,V784C,V784D,V784E,V784F,V784K,V784L,V784M,V784N,V784O,V784X,V791A,V822,V823,V824,V825,V826A,V828,V829,V833A,V834A,V836,V850A,V850B,V852A,V853A,V857A,V857B,V857C,V857D,V858,V801,V802,V803,V804,V805,V811,V812,V813,V814,V815A,V815B,V815C,D005,D101A,D101B,D101C,D101D,D101E,D102,D103A,D103B,D103C,D103D,D103E,D103F,D103G,D104,D105A,D105B,D105C,D105D,D105E,D105F,D105H,D105I,D105J,D105K,D106,D107,D108,D109,D110A,D110B,D110D,D110E,D110F,D110G,D111,D112,D112A,D113,D114,D115B,D115C,D115D,D115F,D115G,D115K,D115L,D115O,D115P,D115Q,D115V,D115W,D115X,D115Y,D115XE,D117A,D118A,D118B,D118C,D118D,D118F,D118G,D118J,D118K,D118L,D118O,D118P,D118Q,D118V,D118W,D118X,D118Y,D118XE,D119H,D119I,D119J,D119K,D119U,D119X,D119Y,D119XB,D119XD,D119XE,D119XF,D119XG,D119XH,D121,D122A,D122B,D122C,D124,D125,D126,D127,D128,D129,D130A,D130B,D130C,S1229AA,S1229AB,S1229AC,S1229BA,S1229BB,S1229BC,S1229BD,S1229BE,S1229BF,S1229BG,S102A,S104BA,S104BB,S104BC,S104BD,S104BZ,S104C,S104P,S108L,S109,S207C,S207D,S208,S239A,S305A,S306A,S308A,S308B,S308C,S705B,S705C,S705D,S717,S723,S815E,S904L,S933,S1107A,S1107B,SEDUC,SPROV,SPEDUC,HHID,HV000,HV001,HV002,HV003,HV004,HV005,HV006,HV007,HV008,HV008A,HV009,HV010,HV012,HV013,HV014,HV015,HV016,HV017,HV018,HV020,HV021,HV022,HV023,HV024,HV025,HV027,HV028,HV030,HV042,HV044,HV045A,HV045B,HV045C,HV046,HV801,HV802,HV803,HV201,HV202,HV201A,HV204,HV205,HV206,HV207,HV208,HV209,HV210,HV211,HV212,HV213,HV214,HV215,HV216,HV217,HV218,HV219,HV220,HV221,HV225,HV226,HV230A,HV230B,HV232,HV232B,HV232Y,HV235,HV237,HV237A,HV237B,HV237C,HV237D,HV237E,HV237F,HV237X,HV237Z,HV238,HV238A,HV241,HV242,HV243A,HV243B,HV243C,HV243D,HV243E,HV244,HV245,HV246,HV246A,HV246B,HV246C,HV246D,HV246E,HV246F,HV246G,HV252,HV270,HV271,HV270A,HV271A,SHUSN,SHSN,SHPROV,SH109B,SH109C,SH121G,SH121H,SH121I,SH121J,SH121K,SH122H,SH123A,SH123B,SH124A,SH201,SH202,SH206,SH207,SH220,SH221A,SH301A,SH301B,SH301C,SH301D,SH301E,SH301F,SH301G,SH301H,SH301I,SH301J,SH301K,SH301L,SH301M,SH301N,SH301O,SH301X,SH301Z,SH302A,SH302B,SH302C,SH302D,SH302E,SH302F,SH302G,SH302H,SH302I,SH302J,SH302K,SH302L,SH302M,SH302N,SH302O,SH302X,SH302Y,SH302Z,SH303,SH304,SH305,SH306,SH307,SH308A,SH308B,SH308C,SH308D,SH308E,SH308F,SH308G,SH308H,SH308X,SH401A,SH401B,SH401C,SH401D,SH401E,SH401F,SH401G,SH401H,SH401I,SH401J,SH401K,SH401L,SH401M,SH401N,

In [25]:
merged_df.columns

Index(['CASEID', 'V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006',
       'V007', 'V008',
       ...
       'SH301G-recoded', 'SH301H-recoded', 'SH301I-recoded', 'SH301J-recoded',
       'SH301K-recoded', 'SH301L-recoded', 'SH301M-recoded', 'SH301N-recoded',
       'SH301O-recoded', 'SH301X-recoded'],
      dtype='object', length=848)

#### <font color= 'pink'> **Practice:**</font> Create a new column `SH301-freq`  that counts the total number of LGU programs the households are aware of

In [26]:
# Insert code here
merged_df['SH301-freq'] = merged_df[['SH301A-recoded','SH301B-recoded','SH301C-recoded','SH301D-recoded','SH301E-recoded','SH301F-recoded',
                                     'SH301G-recoded', 'SH301H-recoded', 'SH301I-recoded', 'SH301J-recoded',
                                     'SH301K-recoded', 'SH301L-recoded', 'SH301M-recoded', 'SH301N-recoded',
                                     'SH301O-recoded', 'SH301X-recoded']].sum(axis = 1)

merged_df.head()

,CASEID,V000,V001,V002,V003,V004,V005,V006,V007,V008,V008A,V009,V010,V011,V012,V013,V014,V015,V016,V018,V019,V019A,V020,V021,V022,V024,V025,V027,V028,V030,V034,V042,V044,V045A,V045B,V045C,V046,V023,V101,V102,V104,V105,V105A,V106,V107,V113,V115,V116,V119,V120,V121,V122,V123,V124,V125,V127,V128,V129,V130,V131,V133,V135,V136,V137,V138,V139,V140,V149,V150,V151,V152,V153,AWFACTT,AWFACTU,AWFACTR,AWFACTE,AWFACTW,V157,V158,V159,V160,V161,V169A,V169B,V170,V171A,V171B,V190,V191,V190A,V191A,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V235,V237,V238,V244,V301,V302A,V312,V313,V315,V316,V317,V318,V319,V320,V321,V322,V325A,V326,V327,V337,V361,V362,V363,V364,V367,V376,V376A,V379,V380,V384A,V384B,V384C,V384D,V393,V393A,V394,V395,V3A00Y,V3A00Z,V3A01,V3A02,V3A03,V3A04,V3A05,V3A06,V3A07,V3A08A,V3A08B,V3A08C,V3A08D,V3A08E,V3A08F,V3A08G,V3A08H,V3A08I,V3A08J,V3A08K,V3A08L,V3A08M,V3A08N,V3A08P,V3A08Q,V3A08R,V3A08S,V3A08T,V3A08U,V3A08V,V3A08X,V3A08Z,V401,V404,V405,V406,V415,V416,V417,V418,V418A,V426,V447,V463A,V463B,V463C,V463D,V463E,V463F,V463G,V463H,V463I,V463X,V463Z,V463AA,V463AB,V464,V465,V467B,V467C,V467D,V467F,V468,V501,V502,V503,V504,V507,V508,V509,V510,V511,V512,V513,V525,V527,V528,V529,V530,V531,V532,V535,V536,V537,V602,V603,V604,V605,V613,V614,V616,V621,V623,V624,V625,V626,V625A,V626A,V627,V628,V629,V632,V632A,V633B,V701,V702,V704,V704A,V705,V714,V714A,V715,V716,V717,V719,V729,V730,V731,V732,V739,V741,V743A,V743B,V743D,V743F,V744A,V744B,V744C,V744D,V744E,V745A,V745B,V746,V750,V751,V754CP,V754DP,V754JP,V754WP,V756,V761,V762,V763A,V763B,V763C,V766A,V766B,V767A,V774A,V774B,V774C,V777A,V781,V783,V784A,V784B,V784C,V784D,V784E,V784F,V784K,V784L,V784M,V784N,V784O,V784X,V791A,V822,V823,V824,V825,V826A,V828,V829,V833A,V834A,V836,V850A,V850B,V852A,V853A,V857A,V857B,V857C,V857D,V858,V801,V802,V803,V804,V805,V811,V812,V813,V814,V815A,V815B,V815C,D005,D101A,D101B,D101C,D101D,D101E,D102,D103A,D103B,D103C,D103D,D103E,D103F,D103G,D104,D105A,D105B,D105C,D105D,D105E,D105F,D105H,D105I,D105J,D105K,D106,D107,D108,D109,D110A,D110B,D110D,D110E,D110F,D110G,D111,D112,D112A,D113,D114,D115B,D115C,D115D,D115F,D115G,D115K,D115L,D115O,D115P,D115Q,D115V,D115W,D115X,D115Y,D115XE,D117A,D118A,D118B,D118C,D118D,D118F,D118G,D118J,D118K,D118L,D118O,D118P,D118Q,D118V,D118W,D118X,D118Y,D118XE,D119H,D119I,D119J,D119K,D119U,D119X,D119Y,D119XB,D119XD,D119XE,D119XF,D119XG,D119XH,D121,D122A,D122B,D122C,D124,D125,D126,D127,D128,D129,D130A,D130B,D130C,S1229AA,S1229AB,S1229AC,S1229BA,S1229BB,S1229BC,S1229BD,S1229BE,S1229BF,S1229BG,S102A,S104BA,S104BB,S104BC,S104BD,S104BZ,S104C,S104P,S108L,S109,S207C,S207D,S208,S239A,S305A,S306A,S308A,S308B,S308C,S705B,S705C,S705D,S717,S723,S815E,S904L,S933,S1107A,S1107B,SEDUC,SPROV,SPEDUC,HHID,HV000,HV001,HV002,HV003,HV004,HV005,HV006,HV007,HV008,HV008A,HV009,HV010,HV012,HV013,HV014,HV015,HV016,HV017,HV018,HV020,HV021,HV022,HV023,HV024,HV025,HV027,HV028,HV030,HV042,HV044,HV045A,HV045B,HV045C,HV046,HV801,HV802,HV803,HV201,HV202,HV201A,HV204,HV205,HV206,HV207,HV208,HV209,HV210,HV211,HV212,HV213,HV214,HV215,HV216,HV217,HV218,HV219,HV220,HV221,HV225,HV226,HV230A,HV230B,HV232,HV232B,HV232Y,HV235,HV237,HV237A,HV237B,HV237C,HV237D,HV237E,HV237F,HV237X,HV237Z,HV238,HV238A,HV241,HV242,HV243A,HV243B,HV243C,HV243D,HV243E,HV244,HV245,HV246,HV246A,HV246B,HV246C,HV246D,HV246E,HV246F,HV246G,HV252,HV270,HV271,HV270A,HV271A,SHUSN,SHSN,SHPROV,SH109B,SH109C,SH121G,SH121H,SH121I,SH121J,SH121K,SH122H,SH123A,SH123B,SH124A,SH201,SH202,SH206,SH207,SH220,SH221A,SH301A,SH301B,SH301C,SH301D,SH301E,SH301F,SH301G,SH301H,SH301I,SH301J,SH301K,SH301L,SH301M,SH301N,SH301O,SH301X,SH301Z,SH302A,SH302B,SH302C,SH302D,SH302E,SH302F,SH302G,SH302H,SH302I,SH302J,SH302K,SH302L,SH302M,SH302N,SH302O,SH302X,SH302Y,SH302Z,SH303,SH304,SH305,SH306,SH307,SH308A,SH308B,SH308C,SH308D,SH308E,SH308F,SH308G,SH308H,SH308X,SH401A,SH401B,SH401C,SH401D,SH401E,SH401F,SH401G,SH401H,SH401I,SH401J,SH401K,SH401L,SH401M,SH401N,

--------------------------------------
## Grouping and Aggregation

Grouping and aggregation are critical components of data analysis which involve:

- **Splitting** data into groups based on some features.
- **Applying** a function to each group independently.
- **Combining** the result into data structure.

#### a. Value counts
- specifies the number of entries per item in the column

In [27]:
# Get number of entries - Source of drinking water (HV201)
merged_df["HV201"].value_counts()

Bottled water                                           9435
Piped into dwelling                                     4691
Tube well or borehole                                   3368
Protected spring                                        1720
Protected well                                          1120
Public tap/standpipe                                    1039
Piped to yard/plot                                      1002
Piped to neighbor                                        889
Unprotected spring                                       768
Unprotected well                                         538
Rainwater                                                201
Tanker truck                                             115
River/dam/lake/ponds/stream/canal/irrigation channel      92
Cart with small tank                                      89
Other                                                      7
Name: HV201, dtype: int64

- we can sort the results using `sort_values`

In [28]:
# Get number of entries sorted in ascending order
merged_df["HV201"].value_counts().sort_values(ascending = True)

Other                                                      7
Cart with small tank                                      89
River/dam/lake/ponds/stream/canal/irrigation channel      92
Tanker truck                                             115
Rainwater                                                201
Unprotected well                                         538
Unprotected spring                                       768
Piped to neighbor                                        889
Piped to yard/plot                                      1002
Public tap/standpipe                                    1039
Protected well                                          1120
Protected spring                                        1720
Tube well or borehole                                   3368
Piped into dwelling                                     4691
Bottled water                                           9435
Name: HV201, dtype: int64

- to sort in descending order, just add `ascending = False` as argument

In [29]:
# Get number of entries sorted in descending order
merged_df["HV201"].value_counts().sort_values(ascending = False)

Bottled water                                           9435
Piped into dwelling                                     4691
Tube well or borehole                                   3368
Protected spring                                        1720
Protected well                                          1120
Public tap/standpipe                                    1039
Piped to yard/plot                                      1002
Piped to neighbor                                        889
Unprotected spring                                       768
Unprotected well                                         538
Rainwater                                                201
Tanker truck                                             115
River/dam/lake/ponds/stream/canal/irrigation channel      92
Cart with small tank                                      89
Other                                                      7
Name: HV201, dtype: int64

#### b. Groupby
- `.size` returns the count of rows
- It can also accept multiple columns

In [30]:
# V101 - Region
merged_df.groupby("V101").size()

V101
Autonomous Region in Muslim Mindanao    1759
Bicol                                   1642
Cagayan Valley                          1067
Calabarzon                              1414
Caraga                                  1426
Central Luzon                           2056
Central Visayas                         1313
Cordillera                              1469
Davao                                   1266
Eastern Visayas                         1595
Ilocos                                   970
Mimaropa                                1257
National Capital                        2574
Northern Mindanao                       1518
Soccsksargen                            1140
Western Visayas                         1742
Zamboanga Peninsula                      866
dtype: int64

How many women with education for each region?

In [31]:
# V101 - Region, V106 - Educational attainment
merged_df.groupby(["V101", "V106"]).size()

V101                                  V106        
Autonomous Region in Muslim Mindanao  Higher           379
                                      No education     107
                                      Primary          580
                                      Secondary        693
Bicol                                 Higher           456
                                      No education       7
                                      Primary          261
                                      Secondary        918
Cagayan Valley                        Higher           449
                                      No education       6
                                      Primary          134
                                      Secondary        478
Calabarzon                            Higher           497
                                      No education       4
                                      Primary          151
                                      Secondary        762
Carag

We can apply another column in brackets after groupby to indicate which column to aggregate.

In [32]:
# V106 - Educational attainment, V201 - Total children ever born
merged_df.groupby("V106")["V201"].mean()

V106
Don't know     5.0000
Higher         1.3106
No education   4.1150
Primary        3.4487
Secondary      1.7340
Name: V201, dtype: float64

#### <font color='pink'> **Practice:**</font> Find the number of women currently pregnant per region. Which region had the most number of pregnant respondents during the time of the survey?

In [33]:
#Insert code here
# V101 - Region, V213 - Currently pregnant
merged_df.groupby(["V101","V213"]).size()

V101                                  V213        
Autonomous Region in Muslim Mindanao  No or unsure    1656
                                      Yes              103
Bicol                                 No or unsure    1578
                                      Yes               64
Cagayan Valley                        No or unsure    1031
                                      Yes               36
Calabarzon                            No or unsure    1356
                                      Yes               58
Caraga                                No or unsure    1356
                                      Yes               70
Central Luzon                         No or unsure    1984
                                      Yes               72
Central Visayas                       No or unsure    1268
                                      Yes               45
Cordillera                            No or unsure    1415
                                      Yes               54
Davao

We can have multiple columns to aggregate. These columns can either be aggregated with the same function or with different functions.

In [34]:
# V101 - Region, V201 - Total children ever born, V012 - Respondent's current age
merged_df.groupby(["V101"])[["V012", "V201"]].mean()

,V012,V201
V101,,
Autonomous Region in Muslim Mindanao,28.9267,2.2166
Bicol,29.6498,2.0956
Cagayan Valley,30.3561,1.8276
Calabarzon,30.5396,1.7546
Caraga,29.9944,1.9923
Central Luzon,30.3103,1.6556
Central Visayas,30.1203,1.7037
Cordillera,29.9278,1.9551
Davao,30.7962,2.1461


Now that we have two columns, if we wish to sort them, we need to add `by` as an argument in sort_values

In [35]:
# V101 - Region, V201 - Total children ever born, V012 - Respondent's current age
merged_df.groupby(["V101"])[["V012", "V201"]].mean().sort_values(by="V201", ascending = False)

,V012,V201
V101,,
Autonomous Region in Muslim Mindanao,28.9267,2.2166
Zamboanga Peninsula,30.5381,2.1640
Davao,30.7962,2.1461
Eastern Visayas,29.8614,2.1122
Mimaropa,30.3246,2.1018
Bicol,29.6498,2.0956
Soccsksargen,30.2588,2.0772
Caraga,29.9944,1.9923
Cordillera,29.9278,1.9551


The aggregated funtion uses the syntax: 
`.agg(new_column_name=(old_column_name, function))`

In [36]:
# V101 - Region, V201 - Total children ever born, V012 - Respondent's current age
# S239A -  Age at first menstrual period
merged_df.groupby(["V101"]).agg(ave_age=("V012", "mean"), ave_num_child=("V201", "mean"), 
                                min_age_1prd=("S239A", "min"), max_age_1prd=("S239A", "max"))

TypeError: '<=' not supported between instances of 'str' and 'float'

In [37]:
merged_df['S239A'].unique()

array(['14.0', 'Don’t know', '12.0', '18.0', '15.0', '13.0', '17.0',
       '16.0', '48.0', '11.0', '10.0', '47.0', '19.0', '29.0', '40.0',
       nan, '8.0', '34.0', '9.0', '20.0', '23.0', '24.0', '35.0', '22.0',
       '21.0', '42.0', '46.0', 12.0, 13.0, 14.0, 10.0, 11.0, 15.0, 16.0,
       17.0, 18.0, 9.0, 44.0, 33.0, 19.0, 30.0, 22.0, 46.0, 34.0, 28.0,
       20.0, '28.0', '31.0', '33.0', '38.0', 41.0, '39.0', '32.0', '45.0',
       '26.0', '43.0', '37.0'], dtype=object)

In [38]:
merged_df = merged_df[merged_df['S239A']!='Don’t know']
merged_df['S239A'] = merged_df['S239A'].astype(float)

merged_df.groupby(["V101"]).agg(ave_age=("V012", "mean"), ave_num_child=("V201", "mean"), 
                                min_age_1prd=("S239A", "min"), max_age_1prd=("S239A", "max"))

<ipython-input-38-f37e78cd1c67>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['S239A'] = merged_df['S239A'].astype(float)


,ave_age,ave_num_child,min_age_1prd,max_age_1prd
V101,,,,
Autonomous Region in Muslim Mindanao,28.9219,2.2132,8.0000,48.0000
Bicol,29.6172,2.0897,8.0000,46.0000
Cagayan Valley,30.3346,1.8289,9.0000,19.0000
Calabarzon,30.5347,1.7535,9.0000,28.0000
Caraga,29.9867,1.9874,9.0000,23.0000
Central Luzon,30.2759,1.6475,8.0000,34.0000
Central Visayas,30.1250,1.7043,9.0000,38.0000
Cordillera,29.9243,1.9523,9.0000,40.0000
Davao,30.7599,2.1339,9.0000,22.0000
